In [1]:
import os, sys
import json
import parseidf
import math
import numpy as np
import pandas as pd
from shapely.geometry import Polygon

In [2]:
# determinant of matrix a
def det(a):
    return a[0][0]*a[1][1]*a[2][2] + a[0][1]*a[1][2]*a[2][0] + a[0][2]*a[1][0]*a[2][1] - a[0][2]*a[1][1]*a[2][0] - a[0][1]*a[1][0]*a[2][2] - a[0][0]*a[1][2]*a[2][1]

#unit normal vector of plane defined by points a, b, and c
def unit_normal(a, b, c):
    x = det([[1,a[1],a[2]],
             [1,b[1],b[2]],
             [1,c[1],c[2]]])
    y = det([[a[0],1,a[2]],
             [b[0],1,b[2]],
             [c[0],1,c[2]]])
    z = det([[a[0],a[1],1],
             [b[0],b[1],1],
             [c[0],c[1],1]])
    magnitude = (x**2 + y**2 + z**2)**.5
    return (x/magnitude, y/magnitude, z/magnitude)

#dot product of vectors a and b
def dot(a, b):
    return a[0]*b[0] + a[1]*b[1] + a[2]*b[2]

#cross product of vectors a and b
def cross(a, b):
    x = a[1] * b[2] - a[2] * b[1]
    y = a[2] * b[0] - a[0] * b[2]
    z = a[0] * b[1] - a[1] * b[0]
    return (x, y, z)

#area of polygon poly
def area(poly):
    if len(poly) < 3: # not a plane - no area
        return 0

    total = [0, 0, 0]
    for i in range(len(poly)):
        vi1 = poly[i]
        if i is len(poly)-1:
            vi2 = poly[0]
        else:
            vi2 = poly[i+1]
        prod = cross(vi1, vi2)
        total[0] += prod[0]
        total[1] += prod[1]
        total[2] += prod[2]
    result = dot(total, unit_normal(poly[0], poly[1], poly[2]))
    return abs(result/2)

In [3]:
df_info = pd.read_csv('scene-json/meta/cityffd_case_info.csv', sep=',',encoding='UTF-8')
df_info.head()

,id,building_type_index,user_defined_building_id,shading_x_shift,shading_y_shift,orientation,x_shift_0,y_shift_0,x_origin,y_origin
0,3981157,2,1172,1832476.84,645387.89,82.0,263.91,153.62,1832212.93,645234.27
1,3981158,2,873,1832572.69,645358.24,81.9,359.76,123.97,1832212.93,645234.27
2,3981159,2,1717,1832578.73,645257.41,82.2,365.80,23.14,1832212.93,645234.27
3,3981160,2,1613,1832474.84,645547.17,82.2,261.91,312.90,1832212.93,645234.27
4,3981161,2,727,1832426.85,645283.09,82.0,213.92,48.82,1832212.93,645234.27


In [4]:
offset = {}
bldg_names = set()
for index, row in df_info.iterrows():
    bldg_name = 'bldg_' + str(int(row['id'])) + '_' + str(int(row['user_defined_building_id']))
    offset[bldg_name] = (row['x_shift_0'], row['y_shift_0'])
    bldg_names.add(bldg_name)

In [5]:
len(bldg_names)

97

In [7]:
work_folder = 'SF_100'
result_folder = 'scene-json'

result_json = {}
result_json['features'] = []
for bldg_id in bldg_names:

    input_file_name = os.path.join(work_folder, bldg_id, 'model.idf')
    x_offset, y_offset = offset[bldg_id][0], offset[bldg_id][1]
    with open(input_file_name, 'r', encoding='utf-8') as f:
        idf = parseidf.parse(f.read())
        result_json_bldg = {}
        result_json_bldg['id'] = bldg_id
        result_json_bldg['properties'] = {}
        result_json_bldg['properties']['objects'] = []
        surf_id = 0
        for idf_obj in idf['BuildingSurface:Detailed'.upper()]:
#             if idf_obj[5] == 'Outdoors':
            if idf_obj[5] == 'Outdoors' or (idf_obj[5] == 'Adiabatic' and idf_obj[2] == 'Wall'):
                surf_id += 1
                surf_name = idf_obj[1].replace(' ', '-').replace('_', '-').replace('--', '-')
                num_vertex = (len(idf_obj) - 11) / 3
                cur_polygon = list()
                z_list = []
                for v_num in range(int(num_vertex)):
                    vertexes = [float(idf_obj[11 + v_num * 3]) + x_offset, 
                                float(idf_obj[12 + v_num * 3]) + y_offset, 
                                float(idf_obj[13 + v_num * 3])]
                    cur_polygon.append(vertexes)
                    z_list.append(float(idf_obj[13 + v_num * 3]))
                curr_poly = Polygon(cur_polygon)
                cur_surf_json = dict()
                cur_surf_json['id'] = 'surf_' + str(surf_id)
                cur_surf_json['type'] = 'surface'
                
                cur_poly_json = dict()
                cur_poly_json['name'] = surf_name
                if sum(z_list)/len(z_list) < 0:
                    print(bldg_id, surf_name)
                    continue
                cur_poly_json['centroid'] = [curr_poly.centroid.x, curr_poly.centroid.y, sum(z_list)/len(z_list)]
#                 cur_poly_json['thermal_emissivity'] = 1
#                 cur_poly_json['solar_absorptance'] = 1
                cur_poly_json['area'] = area(cur_polygon)
                cur_poly_json['v'] = cur_polygon
                cur_surf_json['properties'] = cur_poly_json
                result_json_bldg['properties']['objects'].append(cur_surf_json)
        result_json['features'].append(result_json_bldg)

file = open(os.path.join(result_folder, 'meta' 'sf_dt_98_red_nodes_all_surface_above_ground.json'), 'w')
file.write(json.dumps(result_json, indent=2))
file.close()

Generating LALR tables


bldg_3981191_5464 S1-basement-core-wall-1
bldg_3981191_5464 S1-basement-core-wall-3
bldg_3981175_2698 S1-basement-core-wall-15
bldg_3981170_2986 S1-basement-core-wall-5
bldg_3981170_2986 S1-basement-core-wall-6
bldg_3981170_2986 S1-basement-core-wall-7
bldg_3981161_727 S1-basement-core-wall-1
bldg_3981161_727 S1-basement-core-wall-7
bldg_3981161_727 S1-basement-core-wall-8
bldg_3981161_727 S1-basement-core-wall-9
bldg_3981246_72431 S1-basement-core-wall-4
bldg_3981178_3076 S1-basement-core-wall-0
bldg_3981167_982 S1-basement-core-wall-5
